In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
filepath = '/kaggle/input/suicides-in-india/Suicides in India 2001-2012.csv'
suicide_df = pd.read_csv(filepath)
# suicide_df.drop(columns=['Total'], inplace=True)
suicide_df.tail()

In [ ]:
print(suicide_df['Type_code'].value_counts())
print(suicide_df['Type'].value_counts())
print(suicide_df['Gender'].value_counts())
print(suicide_df['Age_group'].value_counts())
print(suicide_df['State'].value_counts())

In [ ]:
print(suicide_df.isna().sum())

In [ ]:
year_2001 = suicide_df[suicide_df['Year'] == 2001]
year_2001.tail(10)

In [ ]:
year_2001.groupby('Type_code').size()

# Analyzing Suicides in India
We will first try clean the data as the type code messes up. It is not clear if all the rows represent different cases. Let us first analyze causes of suicide.

In [ ]:
causes_df = suicide_df[suicide_df['Type_code'] == 'Causes'].drop(['Type_code'], axis=1)
causes_df.tail()

In [ ]:
top_ten_reasons = causes_df.groupby('Type')['Total'].sum().sort_values(ascending=False)
plt.figure(figsize=(12,16))
sns.barplot(y=top_ten_reasons.index, x= top_ten_reasons.values)

In [ ]:
plt.figure(figsize=(14,12))
sns.lineplot(data=suicide_df.groupby('Year')['Total'].sum())
plt.title('Suicides vs Year', fontsize='15')

In [ ]:
statewise_data = suicide_df.groupby('State')['Total'].sum().sort_values(ascending=False)
statewise_data.drop(index = ['Total (All India)', 'Total (States)', 'Total (Uts)'], inplace=True)
statewise_data = statewise_data.reset_index()
plt.figure(figsize=(12,16))
sns.barplot(y=statewise_data['State'], x=statewise_data['Total'])
plt.title('Suicides in various States', fontsize=15)
print('Yearwise top 3 states')
years = [i for i in range(2001,2013)]
for year in years:
    year_data = suicide_df[suicide_df['Year']== year]
    top_3 = year_data.groupby("State")['Total'].sum().sort_values(ascending=False)
    top_3.drop(index = ['Total (All India)', 'Total (States)', 'Total (Uts)'], inplace=True)
    print(year, top_3[:3], sep='\n')

In [ ]:
gender_data = suicide_df.groupby(['Year','Gender'])['Total'].sum()
gender_data = gender_data.reset_index()
plt.figure(figsize=(12,10))
sns.lineplot(data=gender_data, x='Year', y='Total', hue='Gender')

In [ ]:
plt.figure(figsize=(14,12))
sns.barplot(x=gender_data['Year'], y=gender_data['Total'], hue=gender_data['Gender'])

In [ ]:
cause_by_gender = suicide_df[suicide_df['Type_code'] =='Causes'].groupby(['Type', 'Gender'])['Total'].sum()
cause_by_gender = cause_by_gender.reset_index()
cause_by_gender.replace(to_replace='Bankruptcy or Sudden change in Economic',value='Bankruptcy or Sudden change in Economic Status', inplace=True)
cause_by_gender = cause_by_gender.groupby(['Type','Gender'])['Total'].sum().reset_index()
male_data = cause_by_gender[cause_by_gender['Gender'] == 'Male'].sort_values(by=['Total'], ascending=False)
female_data = cause_by_gender[cause_by_gender['Gender'] == 'Female'].sort_values(by=['Total'], ascending=False)
plt.figure(figsize=(18,14))
plt.subplot(1,2,1)
sns.barplot(y=male_data['Type'], x=male_data['Total']).set_title('Causes of Death For Males')
plt.xlabel('Total Deaths', fontsize=15)
plt.ylabel('Causes', fontsize=15)
plt.subplot(1,2,2)
sns.barplot(y=female_data['Type'], x=female_data['Total']).set_title('Causes of Death For Females')
plt.xlabel('Total Deaths', fontsize=15)
plt.ylabel('')
plt.tight_layout()